In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading data into dataframe
df = pd.read_csv('../input/caravan-insurance-challenge/caravan-insurance-challenge.csv',engine= 'python')
df.head()

In [ ]:
#Having a look at the shape and datatypes of columns of dataframe
df.info()

There are no missing values in the data frame

In [ ]:
#Checking the buying rate
buy = sum(df['CARAVAN'])/len(df.index)*100
buy

The target variable CARAVAN is categorical. Therefore,we will use logistic regression for prediction!

In [ ]:
#Looking at the statistical summary
df.describe()

In [ ]:
#Looking at the train and test data count
df['ORIGIN'].value_counts()


Since, the numbmer of train and test data are more than 100. Hence, we have sufficient amount of data for traing as well as testing.

In [ ]:
#Increasing the number of columns in display
pd.set_option('display.max_columns',100)

In [ ]:
# Checking outliers at 25%, 50%, 75%, 90%, 95% and 99%
df.describe(percentiles=[.25, .5, .75, .90, .95, .99])

From the above table it can be refernced that the there is no outliers in our dataset.<br> We are good to go for Modelling.

In [ ]:
#correlation matrix
plt.figure(figsize=(30,20))
sns.heatmap(df.corr(),annot=True)

It could be observed from the above correlation matrix that there exists some correlated features which will get removed when we use pca.

In [ ]:
#Loading training data in df_train and test data in df_test
df_train = df.loc[df['ORIGIN'] == 'train']
X_train = df_train.drop(['ORIGIN','CARAVAN'], axis=1)
y_train = df_train['CARAVAN']

df_test = df.loc[df['ORIGIN'] == 'test']
X_test = df_test.drop(['ORIGIN','CARAVAN'], axis=1)
y_test = df_test['CARAVAN']

In [ ]:
#Applying MinMax Scaling on the training data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])

In [ ]:
X_train.head()

In [ ]:
X_test.head()

We will be using Principal Component analysis for modelling this data. As it will help us chossing the best princpal componnts with maximum variance information and helps in removing the multicollinearity.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.9) #90% variance is passed as argument

In [ ]:
df_train_pca2 = pca.fit_transform(X_train) #fitting the Training data
df_train_pca2.shape

PCA has chosen 26 features out of 86 features available as the optimal number of principal components. That is great!

In [ ]:
#Following this up, we will model the Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
model_pca = lr.fit(df_train_pca2, y_train) #Fitting on train dataset
df_test_pca2 = pca.transform(X_test) 

In [ ]:
df_test_pca2.shape

The number of attributes choosen for test data set is also 26.

In [ ]:
#Finding the probablities associated with each data point
pred_probs_test2 = model_pca.predict_proba(df_test_pca2)[:,1]

In [ ]:
#Calculating the roc_auc_score for the same model
from sklearn.metrics import roc_auc_score
"{:2.2}".format(roc_auc_score(y_test, pred_probs_test2))

## We got the Roc Score on test data as 72%, which is quite good!